<a href="https://colab.research.google.com/github/mkounkel/Auriga/blob/master/Lab_8_finished.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 8: Density wave

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table,Column
from tqdm import tqdm
import pandas as pd
import plotly.express as px

#defining constants
msun=1.9891e30 # kg
mjup=1.9e27 #kg
au=149597870.691 #m
year=31557600 #s
G=(4*np.pi**2)*au**3/year**2/msun #N m2 kg-2
delta_time=0.001*year*12


Today we will examine objects opening spiral arms in a disk, using the same framework we previously had in the lab on resonances, but extending it to the entire disk. This would work for moons interacting with ring particles, or it would also work for gas giants interacting with protoplanetary disk. For simplicity, we will keep it to be Jupiter and tiny meteoroids throught the plane of the Solar system.

In [ ]:
def create_blank_table(nrows,nparticles=1):
  planet=Table()
  planet['time']=Column(name='time',length=nrows,dtype=float,shape=(nparticles,))
  planet['x']=Column(name='x',length=nrows,dtype=float,shape=(nparticles,))
  planet['y']=Column(name='y',length=nrows,dtype=float,shape=(nparticles,))
  planet['vx']=Column(name='vx',length=nrows,dtype=float,shape=(nparticles,))
  planet['vy']=Column(name='vy',length=nrows,dtype=float,shape=(nparticles,))
  planet['ax']=Column(name='ax',length=nrows,dtype=float,shape=(nparticles,))
  planet['ay']=Column(name='ay',length=nrows,dtype=float,shape=(nparticles,))
  return planet

def defineorbit(period_in_years,theta=0,orbits=100,keep_time=None,nparticles=1):
  period=period_in_years*year
  sma=period_in_years**(2/3.)*au
  v_keplerian=np.sqrt(G*msun/sma)
  acceleration_due_to_sun=G*msun/sma**2

  if keep_time is not None:
    nrows=len(keep_time)
  else:
    nrows=int(period/delta_time*orbits)
  planet=create_blank_table(nrows,nparticles=nparticles)
  planet['x'][0],planet['y'][0]=sma*np.cos(theta),sma*np.sin(theta)
  planet['vx'][0],planet['vy'][0]=-v_keplerian*np.sin(theta),v_keplerian*np.cos(theta)
  planet['ax'][0],planet['ay'][0]=-acceleration_due_to_sun*np.cos(theta),-acceleration_due_to_sun*np.sin(theta)

  return planet

nparticles=10000

planet1=defineorbit(12,orbits=5)
planet2=defineorbit(np.random.uniform(low=2,high=18,size=nparticles),theta=np.random.uniform(low=0,high=2*np.pi,size=nparticles),keep_time=planet1,nparticles=nparticles)
for i in tqdm(range(len(planet1)-1)):
  planet1['time'][i+1]=planet1['time'][i]+delta_time
  planet1['x'][i+1]=planet1['x'][i]+planet1['vx'][i]*delta_time+planet1['ax'][i]*delta_time**2
  planet1['y'][i+1]=planet1['y'][i]+planet1['vy'][i]*delta_time+planet1['ay'][i]*delta_time**2
  planet1['vx'][i+1]=planet1['vx'][i]+planet1['ax'][i]*delta_time
  planet1['vy'][i+1]=planet1['vy'][i]+planet1['ay'][i]*delta_time
  acceleration_due_to_sun=G*msun/(planet1['x'][i+1]**2+planet1['y'][i+1]**2)
  theta=np.arctan2(planet1['y'][i+1],planet1['x'][i+1])
  planet1['ax'][i+1],planet1['ay'][i+1]=-acceleration_due_to_sun*np.cos(theta),-acceleration_due_to_sun*np.sin(theta)

  planet2['time'][i+1]=planet2['time'][i]+delta_time
  planet2['x'][i+1]=planet2['x'][i]+planet2['vx'][i]*delta_time+planet2['ax'][i]*delta_time**2
  planet2['y'][i+1]=planet2['y'][i]+planet2['vy'][i]*delta_time+planet2['ay'][i]*delta_time**2
  planet2['vx'][i+1]=planet2['vx'][i]+planet2['ax'][i]*delta_time
  planet2['vy'][i+1]=planet2['vy'][i]+planet2['ay'][i]*delta_time

  acceleration_due_to_sun=G*msun/(planet2['x'][i+1]**2+planet2['y'][i+1]**2)
  theta=np.arctan2(planet2['y'][i+1],planet2['x'][i+1])
  planet2['ax'][i+1],planet2['ay'][i+1]=-acceleration_due_to_sun*np.cos(theta),-acceleration_due_to_sun*np.sin(theta)

  acceleration_due_to_jupiter=G*mjup/((planet2['x'][i+1]-planet1['x'][i+1])**2+(planet2['y'][i+1]-planet1['y'][i+1])**2)
  theta=np.arctan2((planet2['y'][i+1]-planet1['y'][i+1]),(planet2['x'][i+1]-planet1['x'][i+1]))
  planet2['ax'][i+1],planet2['ay'][i+1]=planet2['ax'][i+1]-acceleration_due_to_jupiter*np.cos(theta),planet2['ay'][i+1]-acceleration_due_to_jupiter*np.sin(theta)



We will then change the data type for our table from astropy tables to pandas - accessing them works generally the same, but the plotting software we will use is more optimized for pandas.

In [ ]:
def makepandas(planet1,planet2,index,rotate=False,calc_orbit=False):
  if rotate:
    theta=getangle(planet1['x'],planet1['y'])
    planet1['x'],planet1['y']=rotation(planet1['x'],planet1['y'],theta)
    planet1['vx'],planet1['vy']=rotation(planet1['vx'],planet1['vy'],theta)
    planet1['ax'],planet1['ay']=rotation(planet1['ax'],planet1['ay'],theta)
    planet2['x'],planet2['y']=rotation(planet2['x'],planet2['y'],theta)
    planet2['vx'],planet2['vy']=rotation(planet2['vx'],planet2['vy'],theta)
    planet2['ax'],planet2['ay']=rotation(planet2['ax'],planet2['ay'],theta)
  df1=pd.DataFrame(data={'x':planet1['x'][index].flatten(),'y':planet1['y'][index].flatten(),'vx':planet1['vx'][index].flatten(),'vy':planet1['vy'][index].flatten(),'time':np.round(planet1['time'][index].flatten()/year,4)})
  df1['type']='planet'
  df1['size']=10

  df2=pd.DataFrame(data={'x':planet2['x'][index].flatten(),'y':planet2['y'][index].flatten(),'vx':planet2['vx'][index].flatten(),'vy':planet2['vy'][index].flatten(),'time':np.round(planet2['time'][index].flatten()/year,4)})
  df2['type']='ring'
  df2['size']=0.1
  if calc_orbit:
    df1=update(df1,[0],len(index))
    df2=update(df2,np.arange(10000),len(index))

  return pd.concat([df2,df1])


index=np.arange(0,5000,200)
df=makepandas(planet1.copy(),planet2.copy(),index,rotate=False,calc_orbit=False)

The code below would make an intearctive plot that would allow you to examine the position of particles at different times.

In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size")

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()


#### Question
- Describe the behavior of the above plot
- How does the structure of particles in the disk evolve after 1 orbital period (which, for this planet is 12 years)?
- How does the structure evolve after 2 orbital periods?
- How does the structure evolve after 3 orbital periods?
- What is it like towards the end of the simulation?

#### Answer

----
Because both the planet is rotating as well as the ring, it may be difficult to keep track of the behavior of the invididual particles. Therefore, let's enter a co-rotating reference frame, i.e., we will keep the position of the planet constant, and observe the motion of particles relative to it.

To do that, we will apply the rotation matrix. We will first find the position angle the planet has relative to the Sun, and we will rotate the system to remove this angle from it.

As a reminder,

$\begin{bmatrix}
x'\\
y'
\end{bmatrix}=\begin{bmatrix}
\cos{\theta} & -\sin{\theta}\\
\sin{\theta} & \cos{\theta}
\end{bmatrix}\begin{bmatrix}
x\\
y
\end{bmatrix}$



In [ ]:
#apply rotation of x & y arrays given angle theta
def rotation(x,y,theta):
  x1=x*np.cos(theta)-y*np.sin(theta)
  y1=x*np.sin(theta)+y*np.cos(theta)
  return x1,y1

#calculate a position angle of a planet given its x & t
def getangle(x,y):
  theta=-np.arctan2(y,x)
  return theta

Now that we have defined these functions, we can turn on a flag in the function that creates our Pandas table and then plot it. The planet should now be stationary.

In [ ]:
index=np.arange(0,5000,200)
df=makepandas(planet1.copy(),planet2.copy(),index,rotate=True,calc_orbit=False)
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size")

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()


#### Question
- Describe qualitative differences between this plot and the one earlier.
- Describe the general motion of the particles
- Reevaluate your assessment of the motion of the particles after 1 year, 2 years, 3 years, and towards the end.

#### Answer

----
While looking at the position of all of these particles is already informative, we should also consider various orbital properties to improve our assessment of the formation of the density wave.

Given that we have a position vector (composed out of x & y arrays) and a velocity vector (composed out of vx & vy arrays), we can calculate instantaneous eccentricity at each step.

To do that, we first need to calculate specific angular momentum ($\vec{h}$, angular momentum per unit mass), and specific orbital energy ($\epsilon$, total orbital energy per unit mass). Keep in mind that since we are not dealing with circular orbits, we have to do full cross product.

$$\vec{h}=\vec{r}\times\vec{v}$$

$$\epsilon=\frac{|\vec{v}|^2}{2}-\frac{GM}{|\vec{r}|}$$

$$e=\sqrt{1+\frac{2ϵ|\vec{h}|^2}{GM}}$$

We can also use the two expressions below to find the semimajor axis $a$, and the position angle of the perihelion $\theta$.

$$|\vec{v}|^2=GM\left(\frac{2}{r}-\frac{1}{a}\right)$$

$$|\vec{r}|=\frac{a(1-e^2)}{1+e\cos{\theta}}$$

In [ ]:
def update(df,init_time,n):
  df['h']=df['x']*df['vy']-df['y']*df['vx']
  df['epsilon']=(df['vx']**2+df['vy']**2)/2-G*msun/np.sqrt(df['x']**2+df['y']**2)
  df['e']=np.sqrt(np.abs(1+2*df['epsilon']*(df['h']/G/msun)**2))
  df['sma']=1/(2/np.sqrt(df['x']**2+df['y']**2)-(df['vx']**2+df['vy']**2)/G/msun)
  df['theta']=(1-df['sma']*(1-df['e']**2)/np.sqrt(df['x']**2+df['y']**2))/df['e']

  df['h_ratio']=df['h']/np.tile(df['h'][init_time],n)
  df['epsilon_ratio']=df['epsilon']/np.tile(df['epsilon'][init_time],n)
  df['sma_ratio']=df['sma']/np.tile(df['sma'][init_time],n)
  return df



Having calculated these factors, we will turn on the flag to make our pandas table.

In [ ]:
index=np.arange(0,5000,200)
df=makepandas(planet1.copy(),planet2.copy(),index,rotate=True,calc_orbit=True)


We will now plot make a plot color-coded by eccentricity.

In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size",color="e",range_color=(0,0.02))
fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()


#### Question
Describe how eccentricity evolves throughout the simulation

#### Answer

---
Color code the plot by the ratio of the specific angular momentum to the initial angular momentum of the particle; pass argument color_continuous_scale=px.colors.diverging.RdYlBu to px.scatter

In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size",color="h_ratio",range_color=(0.995,1.005),color_continuous_scale=px.colors.diverging.RdYlBu)

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()



#### Question
Describe how angular momentum evolves throughout the simulation

#### Answer

----
Color code the plot by the ratio of the specific orbital energy to the initial orbital energy

In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size",color="epsilon_ratio",range_color=(0.995,1.005),color_continuous_scale=px.colors.diverging.RdYlBu)

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()



#### Question
Describe how orbital energy evolves throughout the simulation

#### Answer

---
Color code the plot by the ratio of semi-major axis to the original

In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size",color="sma_ratio",range_color=(0.995,1.005),color_continuous_scale=px.colors.diverging.RdYlBu)

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()



#### Question
Describe how the semimajor axis evolves throughout the simulation

#### Answer

----
Color code the orbit by the position angle of the perihelion. Use argument color_continuous_scale=px.colors.cyclical.mygbm in px.scatter.

In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size",color="theta",range_color=(-np.pi/2,np.pi/2),
                 color_continuous_scale=px.colors.cyclical.mygbm)

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.layout.yaxis.scaleanchor="x"
fig.data[0].update(marker={'line': {'width': 0}})
fig.show()



#### Question
Describe how the position angle of particles changes

#### Answer

----
#### Question
Using all of the plots above interpret how the density wave forms in a disk of particles

#### Answer

----
## Note: this lab has big figures which exceed the maximum file size that Github can handle.

#### Reflection Questions

- How long did this lab take you?
- What were the areas that were easy?
- What were the areas that presented a challenge?

#### Answers

# Extra credit

We will take a more granular look at what is happening close to the formation of the density wave. We will look only at the particles interior to the planet but close to it, starting with the orbital period 9 years, to orbital period 12 years (keeping planet at 12 years). As opposed to the entire ring, we will keep them positioned along an ark with an angle between $-\pi/9$ to $\pi/9$ relative to the planet, itegrating only 0.3 orbit, with delta_time=0.0003\*year\*12

In [ ]:
delta_time=0.0003*year*12
planet1=defineorbit(12,orbits=0.3)
planet2=defineorbit(np.random.uniform(low=9,high=12,size=nparticles),theta=np.random.uniform(low=-np.pi/9,high=np.pi/9,size=nparticles),keep_time=planet1,nparticles=nparticles)
for i in tqdm(range(len(planet1)-1)):
  planet1['time'][i+1]=planet1['time'][i]+delta_time
  planet1['x'][i+1]=planet1['x'][i]+planet1['vx'][i]*delta_time+planet1['ax'][i]*delta_time**2
  planet1['y'][i+1]=planet1['y'][i]+planet1['vy'][i]*delta_time+planet1['ay'][i]*delta_time**2
  planet1['vx'][i+1]=planet1['vx'][i]+planet1['ax'][i]*delta_time
  planet1['vy'][i+1]=planet1['vy'][i]+planet1['ay'][i]*delta_time
  acceleration_due_to_sun=G*msun/(planet1['x'][i+1]**2+planet1['y'][i+1]**2)
  theta=np.arctan2(planet1['y'][i+1],planet1['x'][i+1])
  planet1['ax'][i+1],planet1['ay'][i+1]=-acceleration_due_to_sun*np.cos(theta),-acceleration_due_to_sun*np.sin(theta)

  planet2['time'][i+1]=planet2['time'][i]+delta_time
  planet2['x'][i+1]=planet2['x'][i]+planet2['vx'][i]*delta_time+planet2['ax'][i]*delta_time**2
  planet2['y'][i+1]=planet2['y'][i]+planet2['vy'][i]*delta_time+planet2['ay'][i]*delta_time**2
  planet2['vx'][i+1]=planet2['vx'][i]+planet2['ax'][i]*delta_time
  planet2['vy'][i+1]=planet2['vy'][i]+planet2['ay'][i]*delta_time

  acceleration_due_to_sun=G*msun/(planet2['x'][i+1]**2+planet2['y'][i+1]**2)
  theta=np.arctan2(planet2['y'][i+1],planet2['x'][i+1])
  planet2['ax'][i+1],planet2['ay'][i+1]=-acceleration_due_to_sun*np.cos(theta),-acceleration_due_to_sun*np.sin(theta)

  acceleration_due_to_jupiter=G*mjup/((planet2['x'][i+1]-planet1['x'][i+1])**2+(planet2['y'][i+1]-planet1['y'][i+1])**2)
  theta=np.arctan2((planet2['y'][i+1]-planet1['y'][i+1]),(planet2['x'][i+1]-planet1['x'][i+1]))
  planet2['ax'][i+1],planet2['ay'][i+1]=planet2['ax'][i+1]-acceleration_due_to_jupiter*np.cos(theta),planet2['ay'][i+1]-acceleration_due_to_jupiter*np.sin(theta)
index=np.arange(0,len(planet1),100)
df=makepandas(planet1.copy(),planet2.copy(),index,rotate=True,calc_orbit=True)


In [ ]:
fig = px.scatter(df, x="x", y="y", animation_frame="time",size="size")

fig["layout"].pop("updatemenus")
fig.update_layout(width=700,height=700,autosize=False)
fig.data[0].update(marker={'line': {'width': 0}})
#fig.layout.yaxis.scaleanchor="x"
fig.show()


#### Question
Describe how the interactions between the particles and the planet result in the formation of the density wave
#### Answer